
## Bias drift monitoring with Amazon SageMaker Clarify

This notebook provides a walkthrough of the high level steps involved in monitoring a production ML model with SageMaker Clarify for bias drift. To demonstrate the bias drift monitoring we will use a pre-trained model to deploy an endpoint.  We provide the pre-trained model artifact along with baseline and test datasets along with this notebook.

1. Set up
2. Enable datacapture on a SageMaker endpoint 
3. Generate a baseline with BiasModelMonitor 
4. Schedule continous monitoring to monitor predictions for bias drift on a regular basis.
5. Analyze bias drift monitoring results
6. Clean up

### 1. Set up

#### 1.1. Imports

In [1]:
import copy
import json
import random
import time
import pandas as pd
import os
import boto3
import re

from threading import Thread

from botocore.response import StreamingBody
from sagemaker import get_execution_role, session

from datetime import datetime, timedelta

from sagemaker import get_execution_role, image_uris, Session

from time import gmtime, strftime
from sagemaker.model import Model
from sagemaker.image_uris import retrieve

from sagemaker.clarify import (
    BiasConfig,
    DataConfig,
    ModelConfig,
    ModelPredictedLabelConfig,
    SHAPConfig,
)
from sagemaker.model import Model
from sagemaker.model_monitor import (
    BiasAnalysisConfig,
    CronExpressionGenerator,
    DataCaptureConfig,
    EndpointInput,
    ModelBiasMonitor
)
from sagemaker.s3 import S3Downloader, S3Uploader

#### 1.2 Setup variables

In [2]:
region = boto3.Session().region_name

role = get_execution_role()
print("RoleArn: {}".format(role))

#This is the bucket into which the data is captured
bucket = 'datascience-environment-notebookinstance--06dc7a0224df'
prefix = "BiasDriftAttributionMonitoring" 

data_capture_prefix = "{}/datacapture".format(prefix)
s3_capture_upload_path = "s3://{}/{}".format(bucket, data_capture_prefix)
reports_prefix = "{}/reports".format(prefix)
s3_report_path = "s3://{}/{}".format(bucket, reports_prefix)

ground_truth_upload_path = (
    f"s3://{bucket}/{prefix}/ground_truth_data/{datetime.now():%Y-%m-%d-%H-%M-%S}"
)

print("Capture path: {}".format(s3_capture_upload_path))
print("Report path: {}".format(s3_report_path))


RoleArn: arn:aws:iam::802439482869:role/service-role/AmazonSageMaker-ExecutionRole-20210418T143524
Capture path: s3://datascience-environment-notebookinstance--06dc7a0224df/BiasDriftAttributionMonitoring/datacapture
Report path: s3://datascience-environment-notebookinstance--06dc7a0224df/BiasDriftAttributionMonitoring/reports


#### 1.3 Setup service clients

In [3]:
s3_client = boto3.Session().client("s3")
sagemaker_runtime_client = boto3.Session().client("sagemaker-runtime")
sagemaker_client = boto3.Session().client("sagemaker")

### 2. Enable datacapture on a SageMaker endpoint 

Create an endpoint to showcase the data capture capability in action.

For the endpoint we will use a pre-trained XGBoost model that is ready to deploy. This model was trained in the previous chapters using the weather dataset and has been included in the model directory for ease of use.

Note that you can also train a new model and use your model and data below as well.

#### 2.1 Upload the model object into S3

In [4]:
model_file = open("model/weather-prediction-model.tar.gz", "rb")
s3_key = os.path.join(prefix, "weather-prediction-model.tar.gz")
boto3.Session().resource("s3").Bucket(bucket).Object(s3_key).upload_fileobj(model_file)

#### 2.2  Create SageMaker Model

In [5]:
model_name = f"weather-pred-model-monitor-{datetime.utcnow():%Y-%m-%d-%H%M}"
print("Model name: ", model_name)

model_url = "https://{}.s3-{}.amazonaws.com/{}/weather-prediction-model.tar.gz".format(
    bucket, region, prefix
)

print(model_url)

image_uri = retrieve("xgboost", boto3.Session().region_name, "1.2-1")

model = Model(name=model_name, image_uri=image_uri, model_data=model_url, role=role)

Model name:  weather-pred-model-monitor-2021-08-04-2305
https://datascience-environment-notebookinstance--06dc7a0224df.s3-us-west-2.amazonaws.com/BiasDriftAttributionMonitoring/weather-prediction-model.tar.gz


In [6]:
##Test and validation files to use with model
test_dataset="data/t_file.csv"
validation_dataset = "data/data-drift-baseline-data.csv"
dataset_type = "text/csv"

with open(validation_dataset) as f:
    headers_line = f.readline().rstrip()
    all_headers = headers_line.split(",")
##Get the label name
label_header = all_headers[0]
print(label_header)

value


#### 2.3  Configure datacapture

To enable data capture on the endpoint, you specify the new capture option called `DataCaptureConfig`. On enabling data capture, input to and output from the SageMaker endpoint are captured and saved in S3. Input captured includes the live inference traffic requests and output captured includes predictions from the deployed model.

In [7]:
endpoint_name = "xgb-weather-prediction-model-monitor-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("EndpointName={}".format(endpoint_name))

data_capture_config = DataCaptureConfig(
    enable_capture=True, sampling_percentage=100, destination_s3_uri=s3_capture_upload_path
)

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m4.xlarge",
    endpoint_name=endpoint_name,
    data_capture_config=data_capture_config,
)

EndpointName=xgb-weather-prediction-model-monitor-2021-08-04-23-05-54
-----------------!

#### 2.4 Capture data from endpoint 

This step invokes the endpoint with included sample data for about 3 minutes. Data is captured based on the sampling percentage specified and the capture continues until the data capture option is turned off.

In [8]:
##Use the test file in the data directory  to execute inferences using the test file 't_file.csv' provided
with open('data/t_file.csv', 'r') as TF:
    t_lines = TF.readlines()

In [9]:
### Define a method to run inferences against the endpoint
def get_predictions():
    smrt = boto3.Session().client("sagemaker-runtime")
    #Skip the first line since it has column headers
    for tl in t_lines[1:50]:
        #Remove the first column since it is the label
        test_list = tl.split(",")
        test_list.pop(0)
        test_string = ','.join([str(elem) for elem in test_list])
        
        #print("invoking with payload " + test_string)
    
        result = smrt.invoke_endpoint(EndpointName=endpoint_name,
                                   ContentType="text/csv",
                                   Body=test_string)
        rbody = StreamingBody(raw_stream=result['Body'],content_length=int(result['ResponseMetadata']['HTTPHeaders']['content-length']))
        #print(f"Result from {result['InvokedProductionVariant']} = {rbody.read().decode('utf-8')}")
        print(".", end="", flush=True)
        time.sleep(0.5)

In [10]:
#Get predictions
get_predictions()

.................................................

#### 2.5  View captured data

Now list the data capture files stored in Amazon S3. You should expect to see different files from different time periods organized based on the hour in which the invocation occurred. The format of the Amazon S3 path is:

`s3://{destination-bucket-prefix}/{endpoint-name}/{variant-name}/yyyy/mm/dd/hh/filename.jsonl`

In [11]:
s3_capture_upload_path

's3://datascience-environment-notebookinstance--06dc7a0224df/BiasDriftAttributionMonitoring/datacapture'

In [12]:
#Note : If you see an error in this cell, it could be because the captured files didn't appear in S3 yet.
#Retry after a minute.
current_endpoint_capture_prefix = "{}/{}".format(data_capture_prefix, endpoint_name)

result = s3_client.list_objects(Bucket=bucket, Prefix=current_endpoint_capture_prefix)
capture_files = [capture_file.get("Key") for capture_file in result.get("Contents")]
print("Found Capture Files:")
print("\n ".join(capture_files))

Found Capture Files:
BiasDriftAttributionMonitoring/datacapture/xgb-weather-prediction-model-monitor-2021-08-04-23-05-54/AllTraffic/2021/08/04/23/14-27-166-ffb906f8-243e-42ba-9406-8ad37fbda987.jsonl


Next, view the content of a single capture file. Take a quick peek at the first few lines in the captured file.

In [13]:
def get_obj_body(obj_key):
    return s3_client.get_object(Bucket=bucket, Key=obj_key).get("Body").read().decode("utf-8")


capture_file = get_obj_body(capture_files[-1])
print(capture_file[:2000])

{"captureData":{"endpointInput":{"observedContentType":"text/csv","mode":"INPUT","data":"0,2020,12,4,31,0,19.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0\n","encoding":"CSV"},"endpointOutput":{"observedContentType":"text/csv; charset=utf-8","mode":"OUTPUT","data":"-4.902510643005371","encoding":"CSV"}},"eventMetadata":{"eventId":"147f106c-c75c-46d7-95f1-8f0399b63383","inferenceTime":"2021-08-04T23:14:27Z"},"eventVersion":"0"}
{"captureData":{"endpointInput":{"observedContentType":"text/csv","mode":"INPUT","data":"0,2020,12,4,31,0,19.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0\n","encoding":"CSV"},"endpointOutput":{"observedContentType":"text/csv; charset=utf-8","mode":"OUTPUT","data":"-4.902510643005371","encoding":"CSV"}},"eventMetadata":{"eventId":"644fa091-d5bf-4679-ab77-a80f6c1ab883","inferenceTime":"2021-08-04T23:14:27Z"},"eventVersion":"0"}
{"captureData":{"endpointInput":{"observedContentType":"text/csv","mode":"INPUT","data":"0,2020,12,4,31,0,19.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0\n","enc

Finally, the contents of a single line is present below in a formatted JSON file to observe a little better.

In [14]:
print(json.dumps(json.loads(capture_file.split("\n")[0]), indent=2))

{
  "captureData": {
    "endpointInput": {
      "observedContentType": "text/csv",
      "mode": "INPUT",
      "data": "0,2020,12,4,31,0,19.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0\n",
      "encoding": "CSV"
    },
    "endpointOutput": {
      "observedContentType": "text/csv; charset=utf-8",
      "mode": "OUTPUT",
      "data": "-4.902510643005371",
      "encoding": "CSV"
    }
  },
  "eventMetadata": {
    "eventId": "147f106c-c75c-46d7-95f1-8f0399b63383",
    "inferenceTime": "2021-08-04T23:14:27Z"
  },
  "eventVersion": "0"
}


### 3. Generate a baseline with BiasModelMonitor 

A baselining job runs predictions on validation dataset and suggests constraints. `suggest_baseline()` method starts a `SageMakerClarifyProcessor` processing job using SageMaker Clarify container to generate the constraints.

A bias drift baseline job needs multiple inputs – the data to use for baselining, the sensitive features or facets to check for bias, a model to give predictions and finally a threshold value to indicate when a model prediction is biased. Let’s look at the various configuration objects that capture these details.  

In [15]:
baseline_prefix = prefix + "/baselining"
baseline_data_prefix = baseline_prefix + "/data"
baseline_results_prefix = baseline_prefix + "/results"

baseline_data_uri = f"s3://{bucket}/{baseline_data_prefix}"
baseline_results_uri = f"s3://{bucket}/{baseline_results_prefix}"
model_bias_baselining_job_result_uri = f"{baseline_results_uri}/model_bias"

print(f"Baseline data uri: {baseline_data_uri}")
print(f"Baseline results uri: {baseline_results_uri}")

Baseline data uri: s3://datascience-environment-notebookinstance--06dc7a0224df/BiasDriftAttributionMonitoring/baselining/data
Baseline results uri: s3://datascience-environment-notebookinstance--06dc7a0224df/BiasDriftAttributionMonitoring/baselining/results


#### 3.1 Create ModelBiasMonitor

In [16]:
session = Session()
model_bias_monitor = ModelBiasMonitor(
    role=role,
    sagemaker_session=session,
    max_runtime_in_seconds=1800,
)

#### 3.2 Configure DataConfig

`DataConfig` captures information about the dataset to be analyzed, for example the dataset file, its format (CSV or JSONLines), headers (if any) and label.

In [17]:
model_bias_data_config = DataConfig(
    s3_data_input_path=validation_dataset, #This could also be an S3 path, but using local file for this example
    #s3_data_input_path=baseline_dataset_uri,
    s3_output_path=model_bias_baselining_job_result_uri,
    label=label_header,
    headers=all_headers,
    dataset_type=dataset_type,
)

#### 3.3 Configure BiasConfig

`BiasConfig` is the configuration of the sensitive groups in the dataset you want to check for bias.

In [18]:
##Baseline generation worked with facet name city and values_or_threshold of 100
model_bias_config = BiasConfig(
    label_values_or_threshold=[1],
    #label_values_or_threshold=[100],
    facet_name="city",  
    facet_values_or_threshold=[100]
)

#### 3.4 Configure ModelPredictedLableConfig

`ModelPredictedLabelConfig` specifies how to extract a predicted label from the model output. Since we are using regression model here and the output of the model is the prediction, there is nothing to specify here.  So lets construct an empty object.

In [19]:
#Because this is a regression task, there is nothing to specify here.
model_predicted_label_config = ModelPredictedLabelConfig(
    #probability_threshold=0.8,
    #label='value'
)

#### 3.5 Configure ModelConfig

For bias monitoring, the processing job stands up a shadow endpoint to compute the bias metrics.  `ModelConfig` captures configuration of this model/endpoint.  Once the bias metrics are calculated, the processing will delete the shadow endpoint.

In [20]:
endpoint_instance_count=1

endpoint_instance_type="ml.m4.xlarge"
    
model_config = ModelConfig(
    model_name=model_name,
    instance_count=endpoint_instance_count,
    instance_type=endpoint_instance_type,
    content_type=dataset_type,
    accept_type=dataset_type,
)

#### 3.6 Kick off baselining job

In [21]:
model_bias_monitor.suggest_baseline(
    model_config=model_config,
    data_config=model_bias_data_config,
    bias_config=model_bias_config,
    model_predicted_label_config=model_predicted_label_config,
)
print(f"ModelBiasMonitor baselining job: {model_bias_monitor.latest_baselining_job_name}")


Job Name:  baseline-suggestion-job-2021-08-04-23-18-01-939
Inputs:  [{'InputName': 'dataset', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-west-2-802439482869/baseline-suggestion-job-2021-08-04-23-18-01-939/input/dataset/data-drift-baseline-data.csv', 'LocalPath': '/opt/ml/processing/input/data', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'analysis_config', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://datascience-environment-notebookinstance--06dc7a0224df/BiasDriftAttributionMonitoring/baselining/results/model_bias/analysis_config.json', 'LocalPath': '/opt/ml/processing/input/config', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'analysis_result', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://datascience-environment-notebookinstance--06dc7a0224df/BiasDriftAttributi

Lets wait for the baselining job is completed.

In [23]:
model_bias_monitor.latest_baselining_job.wait(logs=False)

!

Now we can inspect the constraints suggested by the baseline job. 

In [24]:

model_bias_constraints = model_bias_monitor.suggested_constraints()
print()
print(f"ModelBiasMonitor suggested constraints: {model_bias_constraints.file_s3_uri}")
print(S3Downloader.read_file(model_bias_constraints.file_s3_uri))


ModelBiasMonitor suggested constraints: s3://datascience-environment-notebookinstance--06dc7a0224df/BiasDriftAttributionMonitoring/baselining/results/model_bias/analysis.json
{
    "version": "1.0",
    "post_training_bias_metrics": {
        "label": "value",
        "facets": {
            "city": [
                {
                    "value_or_threshold": "(100.0, 2279.0]",
                    "metrics": [
                        {
                            "name": "AD",
                            "description": "Accuracy Difference (AD)",
                            "value": 0.008027100292350653
                        },
                        {
                            "name": "CDDPL",
                            "description": "Conditional Demographic Disparity in Predicted Labels (CDDPL)",
                            "value": null,
                            "error": "Group variable is empty or not provided"
                        },
                        {
      

### 4. Schedule continous monitoring
When that we have the baseline constraints let's analyze and monitor the endpoint on a continuous basis with a Monitoring Schedule

#### 4.1 Generate prediction data for Bias drift  Monitoring

Start generating some artificial traffic.  The cell below starts a thread to send some traffic to the endpoint. Note that you need to stop the kernel to terminate this thread. If there is no traffic, the monitoring jobs are marked as `Failed` since there is no data to process.

In [30]:
from threading import Thread
from time import sleep
import time

#Invoke the endpoint in a loop
def invoke_endpoint_forever():
    while True:
        get_predictions()
        
# Note that you need to stop the kernel to stop the invocations
thread = Thread(target=invoke_endpoint_forever)
thread.start()

...................................................................................................................................................................................

In [25]:
import random


def ground_truth_with_id(inference_id):
    random.seed(inference_id)  # to get consistent results
    rand = random.random()
    return {
        "groundTruthData": {
            #"data": "1" if rand < 0.7 else "0",  # randomly generate positive labels 70% of the time #
             # TODO : Need to make this a decimal??
            "data": rand,
            "encoding": "CSV",
        },
        "eventMetadata": {
            "eventId": str(inference_id),
        },
        "eventVersion": "0",
    }


def upload_ground_truth(records, upload_time):
    fake_records = [json.dumps(r) for r in records]
    data_to_upload = "\n".join(fake_records)
    target_s3_uri = f"{ground_truth_upload_path}/{upload_time:%Y/%m/%d/%H/%M%S}.jsonl"
    print(f"Uploading {len(fake_records)} records to", target_s3_uri)
    S3Uploader.upload_string_as_file_body(data_to_upload, target_s3_uri)

In [26]:
NUM_GROUND_TRUTH_RECORDS = 300


def generate_fake_ground_truth_forever():
    j = 0
    while True:
        fake_records = [ground_truth_with_id(i) for i in range(NUM_GROUND_TRUTH_RECORDS)]
        upload_ground_truth(fake_records, datetime.utcnow())
        j = (j + 1) % 5
        time.sleep(60 * 60)  # do this once an hour


gt_thread = Thread(target=generate_fake_ground_truth_forever)
gt_thread.start()

Uploading 300 records to s3://datascience-environment-notebookinstance--06dc7a0224df/BiasDriftAttributionMonitoring/ground_truth_data/2021-08-04-23-05-41/2021/08/05/00/1937.jsonl


#### 4.2 Create a monitoring schedule

Now that you have the baseline information and ground truth labels, create a monitoring schedule to run model quality monitoring job.

Call `create_monitoring_schedule()` method to schedule a hourly monitor, to analyze the data with monitoring schedule. 

Note that if you did not configure and complete the baseline job, you can use a `BiasAnalysisConfig` parameter of the `create_monitoring_schedule` to pass in the necesaary information about the bias configuation, headers and lables.


In [27]:
schedule_expression = CronExpressionGenerator.hourly()

model_bias_analysis_config = None  ##TODO : Delete this
if not model_bias_monitor.latest_baselining_job:
    model_bias_analysis_config = BiasAnalysisConfig(
        model_bias_config,
        headers=all_headers,
        label=label_header,
    )
    
    
model_bias_monitor.create_monitoring_schedule(
    analysis_config=model_bias_analysis_config,
    output_s3_uri=s3_report_path,
    endpoint_input=EndpointInput(
        endpoint_name=endpoint_name,
        destination="/opt/ml/processing/input/endpoint",
        start_time_offset="-PT1H",
        end_time_offset="-PT0H",
        probability_threshold_attribute=0.8,
    ),
    ground_truth_input=ground_truth_upload_path,
    schedule_cron_expression=schedule_expression,
)
print(f"Model bias monitoring schedule: {model_bias_monitor.monitoring_schedule_name}")

Model bias monitoring schedule: monitoring-schedule-2021-08-05-00-19-41-055


#### 4.3 Wait for the first execution

The schedule starts jobs at the previously specified intervals. Code below wait util time crosses the hour boundary (in UTC) to see executions kick off.

Note: Even for an hourly schedule, Amazon SageMaker has a buffer period of 20 minutes to schedule executions. The execution might start in anywhere from zero to ~20 minutes from the hour boundary. This is expected and done for load balancing in the backend.

In [28]:
def wait_for_execution_to_start(model_monitor):
    print(
        "A hourly schedule was created above and it will kick off executions ON the hour (plus 0 - 20 min buffer)."
    )

    print("Waiting for the first execution to happen", end="")
    schedule_desc = model_monitor.describe_schedule()
    while "LastMonitoringExecutionSummary" not in schedule_desc:
        schedule_desc = model_monitor.describe_schedule()
        print(".", end="", flush=True)
        time.sleep(60)
    print()
    print("Done! Execution has been created")

    print("Now waiting for execution to start", end="")
    while schedule_desc["LastMonitoringExecutionSummary"]["MonitoringExecutionStatus"] in "Pending":
        schedule_desc = model_monitor.describe_schedule()
        print(".", end="", flush=True)
        time.sleep(10)

    print()
    print("Done! Execution has started")

In [29]:
wait_for_execution_to_start(model_bias_monitor)

A hourly schedule was created above and it will kick off executions ON the hour (plus 0 - 20 min buffer).
Waiting for the first execution to happen.............................................
Done! Execution has been created
Now waiting for execution to start
Done! Execution has started


#### 4.4 Wait for the execution to finish

In the previous cell, the first execution has started. This section waits for the execution to finish so that its analysis results are available. Here are the possible terminal states and what each of them mean:

* Completed - This means the monitoring execution completed and no issues were found in the violations report.
* CompletedWithViolations - This means the execution completed, but constraint violations were detected.
* Failed - The monitoring execution failed, maybe due to client error (perhaps incorrect role permissions) or infrastructure issues. Further examination of FailureReason and ExitMessage is necessary to identify what exactly happened.
* Stopped - job exceeded max runtime or was manually stopped.

In [31]:
# Waits for the schedule to have last execution in a terminal status.
def wait_for_execution_to_finish(model_monitor):
    schedule_desc = model_monitor.describe_schedule()
    execution_summary = schedule_desc.get("LastMonitoringExecutionSummary")
    if execution_summary is not None:
        print("Waiting for execution to finish", end="")
        while execution_summary["MonitoringExecutionStatus"] not in [
            "Completed",
            "CompletedWithViolations",
            "Failed",
            "Stopped",
        ]:
            print(".", end="", flush=True)
            time.sleep(60)
            schedule_desc = model_monitor.describe_schedule()
            execution_summary = schedule_desc["LastMonitoringExecutionSummary"]
        print()
        print("Done! Execution has finished")
    else:
        print("Last execution not found")

......

In [32]:
wait_for_execution_to_finish(model_bias_monitor)

Waiting for execution to finish
Done! Execution has finished
......

### 5. Analyze bias drift monitoring results


In [33]:
schedule_desc = model_bias_monitor.describe_schedule()
execution_summary = schedule_desc.get("LastMonitoringExecutionSummary")
execution_summary

{'MonitoringScheduleName': 'monitoring-schedule-2021-08-05-00-19-41-055',
 'ScheduledTime': datetime.datetime(2021, 8, 5, 1, 0, tzinfo=tzlocal()),
 'CreationTime': datetime.datetime(2021, 8, 5, 1, 3, 28, 230000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2021, 8, 5, 1, 3, 32, 22000, tzinfo=tzlocal()),
 'MonitoringExecutionStatus': 'Failed',
 'EndpointName': 'xgb-weather-prediction-model-monitor-2021-08-04-23-05-54',
 'FailureReason': 'Job inputs had no data'}

........

In [34]:

if execution_summary and execution_summary["MonitoringExecutionStatus"] in [
    "Completed",
    "CompletedWithViolations",
]:
    last_model_bias_monitor_execution = model_bias_monitor.list_executions()[-1]
    last_model_bias_monitor_execution_report_uri = (
        last_model_bias_monitor_execution.output.destination
    )
    print(f"Report URI: {last_model_bias_monitor_execution_report_uri}")
    last_model_bias_monitor_execution_report_files = sorted(
        S3Downloader.list(last_model_bias_monitor_execution_report_uri)
    )
    print("Found Report Files:")
    print("\n ".join(last_model_bias_monitor_execution_report_files))
else:
    last_model_bias_monitor_execution = None
    print(
        "====STOP==== \n No completed executions to inspect further. Please wait till an execution completes or investigate previously reported failures."
    )

====STOP==== 
 No completed executions to inspect further. Please wait till an execution completes or investigate previously reported failures.
......

If there are violations compared to the baseline, they will be listed here.

In [35]:
if last_model_bias_monitor_execution:
    model_bias_violations = last_model_bias_monitor_execution.constraint_violations()
    if model_bias_violations:
        print(model_bias_violations.body_dict)

................

### 6.  Cleanup

The endpoint can keep running and capturing data, but if there is no plan to collect more data or use this endpoint further, it should be deleted to avoid incurring additional charges. Note that deleting endpoint does not delete the data that was captured during the model invocations.

##### Stop the monitors scheduled for the endpoint

In [36]:
model_bias_monitor.delete_monitoring_schedule()


Deleting Monitoring Schedule with name: monitoring-schedule-2021-08-05-00-19-41-055
.............................

##### Delete the endpoint

In [ ]:
##Delete the endpoint
response = sagemaker_client.delete_endpoint(
    EndpointName=endpoint_name
)

Please note that the Threads you started to create inference data and ground truth could be still running.  Go ahead and shutdown the kernel to stop them.